# Correct transpilation
Some unexpected noise was appearing in the data. Inspecting the transpiled circuits revealed that the minimal optimization level was contributing to the noise with inconsistent gate decompositions. A higher optimization level yeilded a significant reduction of the noise and a change in the measured model parameters. This could be due to the fact that the transpilation was eliminating the 'id' gates from the circuit. This notebook was intended to explore creating custom transpiler passes to fix this issue. The transpiler turned out to be complicated to work with, and a simpler solution was simply to compile delay instructions into the circuit instead of `id` gates. The backend reports a gate time of 35.5ns, and the delay simulator seems to agree with the id simulation at this time.

In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Pauli
from random import random, choice, choices
from qiskit.providers.fake_provider import FakeQuito
from qiskit.providers.aer import AerSimulator
from qiskit.circuit import Delay

n=2

In [3]:
#noise twirling
def twirled_circuit(n : int, l : int, delay = 35.5) -> QuantumCircuit:
    qc = QuantumCircuit(n)
    ops = [Pauli('I')]*n
    for j in range(l-1):
        for i in range(n):
            p = Pauli(choice(['I','X','Y','Z']))
            op = ops[i].compose(p)
            #leave the first layer with id gates to combine with B gates
            if op.to_label()[-1] == 'I' and j > 0 and not delay == 0:
                qc.append(Delay(delay, "ns"), [i])
            else:
                qc.append(Pauli((op.z,op.x)), [i]) 
            ops[i] = p
        qc.barrier() #separate the layers to prevent further transpiling
    for i in range(n):
        op = ops[i] 
        qc.append(Pauli((op.z,op.x)), [i]) #leave the last layer to combine id gates with B and R gates
    return qc

#readout twirling
def twirled_readout(n):
    qc = QuantumCircuit(n)
    binstr = ""
    for i in range(n):
        r = random()
        if r < .5:
            qc.x(i)
            binstr += '1'
        else:
            qc.id(i)
            binstr += '0'
    return(qc, binstr)

#generate pauli measurement circuitry with corresponding metadata
def twirled_instance(pauli, length, backend, delay = 35.5):
    qc = QuantumCircuit(len(pauli))
    #B gates
    for k,p in enumerate(pauli):
        match p:
            case 'X':
                qc.h(k)
            case 'Y':
                qc.h(k)
                qc.s(k)
            case 'Z':
                qc.id(k)
    #add the twirled layers
    tw = twirled_circuit(2,length, delay)
    qc = qc.compose(tw)
    #switch to measurement basis
    for k,p in enumerate(pauli):
        match p:
            case 'X':
                qc.h(k)
            case 'Y':
                qc.sdg(k)
                qc.h(k)
            case 'Z':
                qc.id(k)
    #add the readout layer
    ro, binstr = twirled_readout(n)
    qc = qc.compose(ro)
    qc.measure_all()

    return qc

## The issue
The following cell shows an example of the undesirable affects of both transpilation levels without the id gates

In [4]:
backend = AerSimulator.from_backend(FakeQuito())
circ = twirled_instance("XX", 3, backend, delay = 0) #render with id gates for demonstration
circ.draw()

┌───┐┌───┐ ░ ┌───┐ ░ ┌───┐┌───┐┌───┐ ░ ┌─┐   
   q_0: ┤ H ├┤ X ├─░─┤ I ├─░─┤ X ├┤ H ├┤ X ├─░─┤M├───
        ├───┤├───┤ ░ ├───┤ ░ ├───┤├───┤├───┤ ░ └╥┘┌─┐
   q_1: ┤ H ├┤ Y ├─░─┤ Y ├─░─┤ I ├┤ H ├┤ X ├─░──╫─┤M├
        └───┘└───┘ ░ └───┘ ░ └───┘└───┘└───┘ ░  ║ └╥┘
meas: 2/════════════════════════════════════════╩══╩═
                                                0  1

In [206]:
transpile(circ,backend,optimization_level=0).draw() #weird compilation effects

global phase: π
               ┌─────────┐┌────┐┌─────────┐   ┌───┐                         »
      q_0 -> 0 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├───┤ I ├─────────────────────────»
               ├─────────┤├────┤├─────────┤┌──┴───┴──┐┌────┐┌────────┐┌────┐»
      q_1 -> 1 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ √X ├┤ Rz(2π) ├┤ √X ├»
               └─────────┘└────┘└─────────┘└─────────┘└────┘└────────┘└────┘»
ancilla_0 -> 2 ─────────────────────────────────────────────────────────────»
                                                                            »
ancilla_1 -> 3 ─────────────────────────────────────────────────────────────»
                                                                            »
ancilla_2 -> 4 ─────────────────────────────────────────────────────────────»
                                                                            »
       meas: 2/═════════════════════════════════════════════════════════════»
                                                                            »
«                            ░    ┌───┐                                      ░ »
«      q_0 -> 0 ─────────────░────┤ I ├──────────────────────────────────────░─»
«               ┌──────────┐ ░ ┌──┴───┴──┐┌────┐┌────────┐┌────┐┌──────────┐ ░ »
«      q_1 -> 1 ┤ Rz(7π/2) ├─░─┤ Rz(π/2) ├┤ √X ├┤ Rz(2π) ├┤ √X ├┤ Rz(7π/2) ├─░─»
«               └──────────┘ ░ └─────────┘└────┘└────────┘└────┘└──────────┘ ░ »
«ancilla_0 -> 2 ───────────────────────────────────────────────────────────────»
«                                                                              »
«ancilla_1 -> 3 ───────────────────────────────────────────────────────────────»
«                                                                              »
«ancilla_2 -> 4 ───────────────────────────────────────────────────────────────»
«                                                                              »
«       meas: 2/═══════════════════════════════════════════════════════════════»
«                                                                              »
«               ┌───┐┌─────────┐┌────┐┌─────────┐┌───┐ ░ ┌─┐   
«      q_0 -> 0 ┤ I ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ I ├─░─┤M├───
«               ├───┤├─────────┤├────┤├─────────┤├───┤ ░ └╥┘┌─┐
«      q_1 -> 1 ┤ I ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ I ├─░──╫─┤M├
«               └───┘└─────────┘└────┘└─────────┘└───┘ ░  ║ └╥┘
«ancilla_0 -> 2 ──────────────────────────────────────────╫──╫─
«                                                         ║  ║ 
«ancilla_1 -> 3 ──────────────────────────────────────────╫──╫─
«                                                         ║  ║ 
«ancilla_2 -> 4 ──────────────────────────────────────────╫──╫─
«                                                         ║  ║ 
«       meas: 2/══════════════════════════════════════════╩══╩═
«                                                         0  1

In [207]:
transpile(circ,backend,optimization_level=1).draw() #looks better, but ID gates are removed

global phase: 3π/2
               ┌─────────┐ ┌────┐┌─────────┐  ░                 ░ ┌─────────┐»
      q_0 -> 0 ┤ Rz(π/2) ├─┤ √X ├┤ Rz(π/2) ├──░─────────────────░─┤ Rz(π/2) ├»
               ├─────────┴┐├────┤├─────────┴┐ ░ ┌────────┐┌───┐ ░ ├─────────┤»
      q_1 -> 1 ┤ Rz(-π/2) ├┤ √X ├┤ Rz(-π/2) ├─░─┤ Rz(-π) ├┤ X ├─░─┤ Rz(π/2) ├»
               └──────────┘└────┘└──────────┘ ░ └────────┘└───┘ ░ └─────────┘»
ancilla_0 -> 2 ──────────────────────────────────────────────────────────────»
                                                                             »
ancilla_1 -> 3 ──────────────────────────────────────────────────────────────»
                                                                             »
ancilla_2 -> 4 ──────────────────────────────────────────────────────────────»
                                                                             »
       meas: 2/══════════════════════════════════════════════════════════════»
                                                                             »
«               ┌────┐┌─────────┐ ░ ┌─┐   
«      q_0 -> 0 ┤ √X ├┤ Rz(π/2) ├─░─┤M├───
«               ├────┤├─────────┤ ░ └╥┘┌─┐
«      q_1 -> 1 ┤ √X ├┤ Rz(π/2) ├─░──╫─┤M├
«               └────┘└─────────┘ ░  ║ └╥┘
«ancilla_0 -> 2 ─────────────────────╫──╫─
«                                    ║  ║ 
«ancilla_1 -> 3 ─────────────────────╫──╫─
«                                    ║  ║ 
«ancilla_2 -> 4 ─────────────────────╫──╫─
«                                    ║  ║ 
«       meas: 2/═════════════════════╩══╩═
«                                    0  1

## Noisy simulation comparison
This shows a comparison of simulating noisy `id` gates and noisy `delay[35.5ns]` gates. Due to the assumption that thermal relaxation errors will dominate during idle period, qubits are initialized to 1, and then the increasing populations of '0' are graphed against circuit depth.

In [5]:
circs_delay = []
circs_id = []
for i in range(5,10):
    qc = QuantumCircuit(1,1)
    qc.x(0)
    qc2 = qc.copy()
    for j in range(2**i):
        qc2.id(0)
        qc.append(Delay(35.5, "ns"), [0])

    qc.measure(0,0)
    qc2.measure(0,0)
    circs_delay.append(qc)
    circs_id.append(qc2)

In [6]:
from qiskit.visualization import plot_histogram
results_delay = []
results_id = []
for circ1, circ2 in zip(circs_delay, circs_id):
    results_delay.append(backend.run(circ1).result().get_counts()['0'])
    results_id.append(backend.run(circ2).result().get_counts()['0'])

In [7]:
print("Results with id gate: ", results_id)
print("Results with equivalent delay: ", results_delay)

Results with id gate:  [103, 108, 144, 246, 344]
Results with equivalent delay:  [90, 118, 152, 260, 355]


In [8]:
#backend reports a single-qubit gate time of 35.5ns
backend.properties().to_dict()['gates']

[{'qubits': [0],
  'gate': 'id',
  'parameters': [{'date': datetime.datetime(2021, 3, 15, 0, 14, 56, tzinfo=tzoffset(None, -14400)),
    'name': 'gate_error',
    'unit': '',
    'value': 0.00025870026697239005},
   {'date': datetime.datetime(2021, 3, 15, 0, 39, 13, tzinfo=tzoffset(None, -14400)),
    'name': 'gate_length',
    'unit': 'ns',
    'value': 35.55555555555556}],
  'name': 'id0'},
 {'qubits': [1],
  'gate': 'id',
  'parameters': [{'date': datetime.datetime(2021, 3, 15, 0, 14, 56, tzinfo=tzoffset(None, -14400)),
    'name': 'gate_error',
    'unit': '',
    'value': 0.002317246824118454},
   {'date': datetime.datetime(2021, 3, 15, 0, 39, 13, tzinfo=tzoffset(None, -14400)),
    'name': 'gate_length',
    'unit': 'ns',
    'value': 35.55555555555556}],
  'name': 'id1'},
 {'qubits': [2],
  'gate': 'id',
  'parameters': [{'date': datetime.datetime(2021, 3, 15, 0, 14, 56, tzinfo=tzoffset(None, -14400)),
    'name': 'gate_error',
    'unit': '',
    'value': 0.0015987430774616644}

# Take two
I still haven't figured out how to ask the transpiler to retain the identity gates but transpile everything else. The only other solution I can think of is to pre-transpile the single-qubit gates used in the circuit and sample from those instead.

In [23]:
from qiskit.quantum_info import pauli_basis
from qiskit.circuit.library import IGate, HGate, SGate
from itertools import product

gate_dict = {}
gate_dict_original = {}

for p in "IXYZ":
    qc = QuantumCircuit(1)
    qc.append(Pauli(p),[0])
    gate_dict[p] = transpile(qc, basis_gates=backend._basis_gates())
    gate_dict_original[p] = qc

gate_dict['I'].id(0)
gate_dict['Z'].id(0)

for h,p in product(["H","HS"],"IXYZ"):
    qc = QuantumCircuit(1)
    match h:
        case "H":
            qc.h(0)
        case "HS":
            qc.h(0)
            qc.s(0)
        case "I":
            qc.id(0)
    qc.append(Pauli(p),[0])
    gate_dict[h+p] = transpile(qc, basis_gates = backend._basis_gates())
    gate_dict_original[h+p] = qc

for p,h,x in product("IXYZ", ["H", "SdgH"], ["","X"]):
    qc = QuantumCircuit(1)
    qc.append(Pauli(p),[0])
    match h:
        case "H":
            qc.h(0)
        case "SdgH":
            qc.sdg(0)
            qc.h(0)
    if x=="X":
        qc.x(0)

    gate_dict[p+h+x] = transpile(qc, basis_gates = backend._basis_gates())
    gate_dict_original[p+h+x] = qc

print(gate_dict.keys())

dict_keys(['I', 'X', 'Y', 'Z', 'HI', 'HX', 'HY', 'HZ', 'HSI', 'HSX', 'HSY', 'HSZ', 'IH', 'IHX', 'ISdgH', 'ISdgHX', 'XH', 'XHX', 'XSdgH', 'XSdgHX', 'YH', 'YHX', 'YSdgH', 'YSdgHX', 'ZH', 'ZHX', 'ZSdgH', 'ZSdgHX'])


In [24]:
qc = QuantumCircuit(len(gate_dict))
for i,p in enumerate(gate_dict.keys()):
    qc = qc.compose(gate_dict[p], [i])


qc2 = QuantumCircuit(len(gate_dict))
for i,p in enumerate(gate_dict.keys()):
    qc2 = qc2.compose(gate_dict_original[p], [i])

qc.barrier()
qc = qc.compose(qc2)
qc.draw()

global phase: 3π/2
         ┌───┐                             ░ ┌───┐                 
 q_0: ───┤ I ├─────────────────────────────░─┤ I ├─────────────────
         ├───┤                             ░ ├───┤                 
 q_1: ───┤ X ├─────────────────────────────░─┤ X ├─────────────────
       ┌─┴───┴──┐    ┌───┐                 ░ ├───┤                 
 q_2: ─┤ Rz(-π) ├────┤ X ├─────────────────░─┤ Y ├─────────────────
       ├───────┬┘    ├───┤                 ░ ├───┤                 
 q_3: ─┤ Rz(π) ├─────┤ I ├─────────────────░─┤ Z ├─────────────────
      ┌┴───────┴┐    ├───┴┐   ┌─────────┐  ░ ├───┤ ┌───┐           
 q_4: ┤ Rz(π/2) ├────┤ √X ├───┤ Rz(π/2) ├──░─┤ H ├─┤ I ├───────────
      ├─────────┴┐   ├────┤   ├─────────┤  ░ ├───┤ ├───┤           
 q_5: ┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(π/2) ├──░─┤ H ├─┤ X ├───────────
      ├──────────┤   ├────┤   ├─────────┴┐ ░ ├───┤ ├───┤           
 q_6: ┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(-π/2) ├─░─┤ H ├─┤ Y ├───────────
      ├─────────┬┘   ├────┤   ├──────────┤ ░ ├───┤ ├───┤           
 q_7: ┤ Rz(π/2) ├────┤ √X ├───┤ Rz(-π/2) ├─░─┤ H ├─┤ Z ├───────────
      ├─────────┤    ├────┤   └┬────────┬┘ ░ ├───┤ ├───┤ ┌───┐     
 q_8: ┤ Rz(π/2) ├────┤ √X ├────┤ Rz(-π) ├──░─┤ H ├─┤ S ├─┤ I ├─────
      ├─────────┴┐   ├────┤    └────────┘  ░ ├───┤ ├───┤ ├───┤     
 q_9: ┤ Rz(-π/2) ├───┤ √X ├────────────────░─┤ H ├─┤ S ├─┤ X ├─────
      ├──────────┤   ├────┤    ┌────────┐  ░ ├───┤ ├───┤ ├───┤     
q_10: ┤ Rz(-π/2) ├───┤ √X ├────┤ Rz(-π) ├──░─┤ H ├─┤ S ├─┤ Y ├─────
      ├─────────┬┘   ├────┤    └────────┘  ░ ├───┤ ├───┤ ├───┤     
q_11: ┤ Rz(π/2) ├────┤ √X ├────────────────░─┤ H ├─┤ S ├─┤ Z ├─────
      ├─────────┤    ├────┤   ┌─────────┐  ░ ├───┤ ├───┤ └───┘     
q_12: ┤ Rz(π/2) ├────┤ √X ├───┤ Rz(π/2) ├──░─┤ I ├─┤ H ├───────────
      ├─────────┴┐   ├────┤   ├─────────┤  ░ ├───┤ ├───┤ ┌───┐     
q_13: ┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(π/2) ├──░─┤ I ├─┤ H ├─┤ X ├─────
      └──┬────┬──┘┌──┴────┴─┐ └─────────┘  ░ ├───┤┌┴───┴┐├───┤     
q_14: ───┤ √X ├───┤ Rz(π/2) ├──────────────░─┤ I ├┤ Sdg ├┤ H ├─────
       ┌─┴────┴─┐ └──┬────┬─┘ ┌─────────┐  ░ ├───┤├─────┤├───┤┌───┐
q_15: ─┤ Rz(-π) ├────┤ √X ├───┤ Rz(π/2) ├──░─┤ I ├┤ Sdg ├┤ H ├┤ X ├
      ┌┴────────┤    ├────┤   ├─────────┴┐ ░ ├───┤└┬───┬┘└───┘└───┘
q_16: ┤ Rz(π/2) ├────┤ √X ├───┤ Rz(-π/2) ├─░─┤ X ├─┤ H ├───────────
      ├─────────┴┐   ├────┤   ├──────────┤ ░ ├───┤ ├───┤ ┌───┐     
q_17: ┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(-π/2) ├─░─┤ X ├─┤ H ├─┤ X ├─────
      └┬────────┬┘   ├────┤   ├──────────┤ ░ ├───┤┌┴───┴┐├───┤     
q_18: ─┤ Rz(-π) ├────┤ √X ├───┤ Rz(-π/2) ├─░─┤ X ├┤ Sdg ├┤ H ├─────
       └─┬────┬─┘ ┌──┴────┴──┐└──────────┘ ░ ├───┤├─────┤├───┤┌───┐
q_19: ───┤ √X ├───┤ Rz(-π/2) ├─────────────░─┤ X ├┤ Sdg ├┤ H ├┤ X ├
      ┌──┴────┴──┐└──┬────┬──┘┌──────────┐ ░ ├───┤└┬───┬┘└───┘└───┘
q_20: ┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(-π/2) ├─░─┤ Y ├─┤ H ├───────────
      ├─────────┬┘   ├────┤   ├──────────┤ ░ ├───┤ ├───┤ ┌───┐     
q_21: ┤ Rz(π/2) ├────┤ √X ├───┤ Rz(-π/2) ├─░─┤ Y ├─┤ H ├─┤ X ├─────
      └──┬────┬─┘ ┌──┴────┴──┐└──────────┘ ░ ├───┤┌┴───┴┐├───┤     
q_22: ───┤ √X ├───┤ Rz(-π/2) ├─────────────░─┤ Y ├┤ Sdg ├┤ H ├─────
       ┌─┴────┴─┐ └──┬────┬──┘┌──────────┐ ░ ├───┤├─────┤├───┤┌───┐
q_23: ─┤ Rz(-π) ├────┤ √X ├───┤ Rz(-π/2) ├─░─┤ Y ├┤ Sdg ├┤ H ├┤ X ├
      ┌┴────────┴┐   ├────┤   ├─────────┬┘ ░ ├───┤└┬───┬┘└───┘└───┘
q_24: ┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(π/2) ├──░─┤ Z ├─┤ H ├───────────
      ├─────────┬┘   ├────┤   ├─────────┤  ░ ├───┤ ├───┤ ┌───┐     
q_25: ┤ Rz(π/2) ├────┤ √X ├───┤ Rz(π/2) ├──░─┤ Z ├─┤ H ├─┤ X ├─────
      └┬────────┤    ├────┤   ├─────────┤  ░ ├───┤┌┴───┴┐├───┤     
q_26: ─┤ Rz(-π) ├────┤ √X ├───┤ Rz(π/2) ├──░─┤ Z ├┤ Sdg ├┤ H ├─────
       └─┬────┬─┘ ┌──┴────┴─┐ └─────────┘  ░ ├───┤├─────┤├───┤┌───┐
q_27: ───┤ √X ├───┤ Rz(π/2) ├──────────────░─┤ Z ├┤ Sdg ├┤ H ├┤ X ├
         └────┘   └─────────┘              ░ └───┘└─────┘└───┘└───┘

In [70]:
#generate pauli measurement circuitry with corresponding metadata
def twirled_instance(pauli, length):
    n= len(pauli)
    circ_string = [] 
    #choose first layer
    twirl_layer = choices(pauli_basis(1, pauli_list = True), k=n)
    #Choose B gates
    circ_string.append([])
    for i,(b,p) in enumerate(zip(pauli, twirl_layer)):
        gate_name = "";
        gate_name = {'X':"H", 'Y':"SH"}.get(b,"")
        gate_name+=p.to_label()
        circ_string[-1].append(gate_name)

    #add the twirled layers
    for l in range(length-1):
        circ_string.append([])
        for i in range(n):
            tw_op = choice("IXYZ")
            circ_string[-1].append(tw_op)
            twirl_layer[i] = twirl_layer[i].compose(Pauli(tw_op))
    #switch to measurement basis
    binstr = choices('01',k=n)
    circ_string.append([])
    for i,(p,b,x) in enumerate(zip(twirl_layer, pauli, binstr)):
        gate_name = p.to_label()[-1]
        gate_name += {'X':'H', 'Y':"SdgH"}.get(b,"")
        gate_name += {'0':'', '1':'X'}[x]
        circ_string[-1].append(gate_name)

    return circ_string

def instance_to_circuit(instance, dictionary):
    n = len(instance[0])
    qc = QuantumCircuit(n,n)
    for layer in instance:
        for i,gate in enumerate(layer):
            qc = qc.compose(dictionary[gate],[i])
        qc.barrier()
    qc.measure(range(n), range(n))
    return qc

In [71]:
qc = instance_to_circuit(twirled_instance("XX", 5), gate_dict)

In [75]:
qc.draw()

global phase: π
               ┌──────────┐┌────┐┌──────────┐ ░ ┌────────┐      ░ ┌────────┐»
      q_0 -> 0 ┤ Rz(-π/2) ├┤ √X ├┤ Rz(-π/2) ├─░─┤ Rz(-π) ├──────░─┤ Rz(-π) ├»
               ├─────────┬┘├────┤├──────────┤ ░ ├────────┤┌───┐ ░ ├────────┤»
      q_1 -> 1 ┤ Rz(π/2) ├─┤ √X ├┤ Rz(-π/2) ├─░─┤ Rz(-π) ├┤ X ├─░─┤ Rz(-π) ├»
               └─────────┘ └────┘└──────────┘ ░ └────────┘└───┘ ░ └────────┘»
ancilla_0 -> 2 ─────────────────────────────────────────────────────────────»
                                                                            »
ancilla_1 -> 3 ─────────────────────────────────────────────────────────────»
                                                                            »
ancilla_2 -> 4 ─────────────────────────────────────────────────────────────»
                                                                            »
          c: 2/═════════════════════════════════════════════════════════════»
                                                                            »
«                     ░  ░ ┌────────┐┌───┐ ░ ┌──────────┐┌────┐┌─────────┐ ░ »
«      q_0 -> 0 ──────░──░─┤ Rz(-π) ├┤ X ├─░─┤ Rz(-π/2) ├┤ √X ├┤ Rz(π/2) ├─░─»
«               ┌───┐ ░  ░ └────────┘└───┘ ░ ├──────────┤├────┤├─────────┤ ░ »
«      q_1 -> 1 ┤ X ├─░──░─────────────────░─┤ Rz(-π/2) ├┤ √X ├┤ Rz(π/2) ├─░─»
«               └───┘ ░  ░                 ░ └──────────┘└────┘└─────────┘ ░ »
«ancilla_0 -> 2 ─────────────────────────────────────────────────────────────»
«                                                                            »
«ancilla_1 -> 3 ─────────────────────────────────────────────────────────────»
«                                                                            »
«ancilla_2 -> 4 ─────────────────────────────────────────────────────────────»
«                                                                            »
«          c: 2/═════════════════════════════════════════════════════════════»
«                                                                            »
«               ┌─┐   
«      q_0 -> 0 ┤M├───
«               └╥┘┌─┐
«      q_1 -> 1 ─╫─┤M├
«                ║ └╥┘
«ancilla_0 -> 2 ─╫──╫─
«                ║  ║ 
«ancilla_1 -> 3 ─╫──╫─
«                ║  ║ 
«ancilla_2 -> 4 ─╫──╫─
«                ║  ║ 
«          c: 2/═╩══╩═
«                0  1

In [73]:
backend.run(qc).result().get_counts()

{'11': 17, '10': 2, '00': 68, '01': 937}

In [127]:
class learn_single_gate_layer:

    def __init__(self, n, backend):
        self.n = n
        self.circuit_string = [[""]*n]
        self.backend = backend
        self.metadata = {"dim":n, "backend":backend.name()}
        self.transpiled_gates = {}
        self.raw_gates = {}
        self._generate_gate_dict(backend)

    def _generate_gate_dict(self, backend):
        transpiled_gates = {}
        raw_gates = {}

        for p in "IXYZ":
            qc = QuantumCircuit(1)
            qc.append(Pauli(p),[0])
            transpiled_gates[p] = transpile(qc, basis_gates=backend._basis_gates())
            raw_gates[p] = qc

        transpiled_gates['I'].id(0)
        transpiled_gates['Z'].id(0)

        for h,p in product(["H","HS"],"IXYZ"):
            qc = QuantumCircuit(1)
            match h:
                case "H":
                    qc.h(0)
                case "HS":
                    qc.h(0)
                    qc.s(0)
                case "I":
                    qc.id(0)
            qc.append(Pauli(p),[0])
            transpiled_gates[h+p] = transpile(qc, basis_gates = backend._basis_gates())
            raw_gates[h+p] = qc

        for p,h,x in product("IXYZ", ["H", "SdgH"], ["","X"]):
            qc = QuantumCircuit(1)
            qc.append(Pauli(p),[0])
            match h:
                case "H":
                    qc.h(0)
                case "SdgH":
                    qc.sdg(0)
                    qc.h(0)
            if x=="X":
                qc.x(0)

            transpiled_gates[p+h+x] = transpile(qc, basis_gates = backend._basis_gates())
            raw_gates[p+h+x] = qc      

        self.raw_gates = raw_gates
        self.transpiled_gates = transpiled_gates

    def print_gate_conversions(self):
        qc = QuantumCircuit(len(gate_dict))
        for i,p in enumerate(gate_dict.keys()):
            qc = qc.compose(gate_dict[p], [i])


            qc2 = QuantumCircuit(len(gate_dict))

        for i,p in enumerate(gate_dict.keys()):
            qc2 = qc2.compose(gate_dict_original[p], [i])

        qc.barrier()
        qc = qc.compose(qc2)
        return qc
    
    #generate pauli measurement circuitry with corresponding metadata
    def generate_instance(self, basis_operator, noise_repetitions):

        if len(basis_operator) != self.n:
            raise Exception("Pauli operator has wrong dimension")

        circ_string = [] 
        #choose first layer
        twirl_layer = choices(pauli_basis(1, pauli_list = True), k=self.n)
        #Choose B gates
        circ_string.append([])
        for i,(b,p) in enumerate(zip(basis_operator, twirl_layer)):
            gate_name = "";
            gate_name = {'X':"H", 'Y':"SH"}.get(b,"")
            gate_name+=p.to_label()
            circ_string[-1].append(gate_name)

        #add the twirled layers
        for l in range(noise_repetitions-1):
            circ_string.append([])
            for i in range(n):
                tw_op = choice("IXYZ")
                circ_string[-1].append(tw_op)
                twirl_layer[i] = twirl_layer[i].compose(Pauli(tw_op))
        #switch to measurement basis
        binstr = choices('01',k=n)
        circ_string.append([])
        for i,(p,b,x) in enumerate(zip(twirl_layer, basis_operator, binstr)):
            gate_name = p.to_label()[-1]
            gate_name += {'X':'H', 'Y':"SdgH"}.get(b,"")
            gate_name += {'0':'', '1':'X'}[x]
            circ_string[-1].append(gate_name)

        self.metadata['rostring'] = binstr
        self.circuit_string = circ_string

    def instance_to_circuit(self, transpiled = True):
        n = self.n
        instance = self.circuit_string

        if transpiled:
            dictionary = self.transpiled_gates
        else:
            dictionary = self.raw_gates

        qc = QuantumCircuit(n,n)
        for layer in instance:
            for i,gate in enumerate(layer):
                qc = qc.compose(dictionary[gate],[i])
            qc.barrier()
        qc.measure(range(n), range(n))
        return qc

In [128]:
procedure = learn_single_gate_layer(2, backend)

In [129]:
procedure.generate_instance("XX", 2)

In [130]:
procedure.instance_to_circuit(transpiled=False).draw()

┌───┐┌───┐ ░ ┌───┐ ░ ┌───┐┌───┐┌───┐ ░ ┌─┐   
q_0: ┤ H ├┤ X ├─░─┤ X ├─░─┤ I ├┤ H ├┤ X ├─░─┤M├───
     ├───┤├───┤ ░ ├───┤ ░ ├───┤├───┤├───┤ ░ └╥┘┌─┐
q_1: ┤ H ├┤ I ├─░─┤ I ├─░─┤ I ├┤ H ├┤ X ├─░──╫─┤M├
     └───┘└───┘ ░ └───┘ ░ └───┘└───┘└───┘ ░  ║ └╥┘
c: 2/════════════════════════════════════════╩══╩═
                                             0  1

In [131]:
procedure.print_gate_conversions().draw()

global phase: 3π/2
         ┌───┐                             ░ ┌───┐                 
 q_0: ───┤ I ├─────────────────────────────░─┤ I ├─────────────────
         ├───┤                             ░ ├───┤                 
 q_1: ───┤ X ├─────────────────────────────░─┤ X ├─────────────────
       ┌─┴───┴──┐    ┌───┐                 ░ ├───┤                 
 q_2: ─┤ Rz(-π) ├────┤ X ├─────────────────░─┤ Y ├─────────────────
       ├───────┬┘    ├───┤                 ░ ├───┤                 
 q_3: ─┤ Rz(π) ├─────┤ I ├─────────────────░─┤ Z ├─────────────────
      ┌┴───────┴┐    ├───┴┐   ┌─────────┐  ░ ├───┤ ┌───┐           
 q_4: ┤ Rz(π/2) ├────┤ √X ├───┤ Rz(π/2) ├──░─┤ H ├─┤ I ├───────────
      ├─────────┴┐   ├────┤   ├─────────┤  ░ ├───┤ ├───┤           
 q_5: ┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(π/2) ├──░─┤ H ├─┤ X ├───────────
      ├──────────┤   ├────┤   ├─────────┴┐ ░ ├───┤ ├───┤           
 q_6: ┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(-π/2) ├─░─┤ H ├─┤ Y ├───────────
      ├─────────┬┘   ├────┤   ├──────────┤ ░ ├───┤ ├───┤           
 q_7: ┤ Rz(π/2) ├────┤ √X ├───┤ Rz(-π/2) ├─░─┤ H ├─┤ Z ├───────────
      ├─────────┤    ├────┤   └┬────────┬┘ ░ ├───┤ ├───┤ ┌───┐     
 q_8: ┤ Rz(π/2) ├────┤ √X ├────┤ Rz(-π) ├──░─┤ H ├─┤ S ├─┤ I ├─────
      ├─────────┴┐   ├────┤    └────────┘  ░ ├───┤ ├───┤ ├───┤     
 q_9: ┤ Rz(-π/2) ├───┤ √X ├────────────────░─┤ H ├─┤ S ├─┤ X ├─────
      ├──────────┤   ├────┤    ┌────────┐  ░ ├───┤ ├───┤ ├───┤     
q_10: ┤ Rz(-π/2) ├───┤ √X ├────┤ Rz(-π) ├──░─┤ H ├─┤ S ├─┤ Y ├─────
      ├─────────┬┘   ├────┤    └────────┘  ░ ├───┤ ├───┤ ├───┤     
q_11: ┤ Rz(π/2) ├────┤ √X ├────────────────░─┤ H ├─┤ S ├─┤ Z ├─────
      ├─────────┤    ├────┤   ┌─────────┐  ░ ├───┤ ├───┤ └───┘     
q_12: ┤ Rz(π/2) ├────┤ √X ├───┤ Rz(π/2) ├──░─┤ I ├─┤ H ├───────────
      ├─────────┴┐   ├────┤   ├─────────┤  ░ ├───┤ ├───┤ ┌───┐     
q_13: ┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(π/2) ├──░─┤ I ├─┤ H ├─┤ X ├─────
      └──┬────┬──┘┌──┴────┴─┐ └─────────┘  ░ ├───┤┌┴───┴┐├───┤     
q_14: ───┤ √X ├───┤ Rz(π/2) ├──────────────░─┤ I ├┤ Sdg ├┤ H ├─────
       ┌─┴────┴─┐ └──┬────┬─┘ ┌─────────┐  ░ ├───┤├─────┤├───┤┌───┐
q_15: ─┤ Rz(-π) ├────┤ √X ├───┤ Rz(π/2) ├──░─┤ I ├┤ Sdg ├┤ H ├┤ X ├
      ┌┴────────┤    ├────┤   ├─────────┴┐ ░ ├───┤└┬───┬┘└───┘└───┘
q_16: ┤ Rz(π/2) ├────┤ √X ├───┤ Rz(-π/2) ├─░─┤ X ├─┤ H ├───────────
      ├─────────┴┐   ├────┤   ├──────────┤ ░ ├───┤ ├───┤ ┌───┐     
q_17: ┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(-π/2) ├─░─┤ X ├─┤ H ├─┤ X ├─────
      └┬────────┬┘   ├────┤   ├──────────┤ ░ ├───┤┌┴───┴┐├───┤     
q_18: ─┤ Rz(-π) ├────┤ √X ├───┤ Rz(-π/2) ├─░─┤ X ├┤ Sdg ├┤ H ├─────
       └─┬────┬─┘ ┌──┴────┴──┐└──────────┘ ░ ├───┤├─────┤├───┤┌───┐
q_19: ───┤ √X ├───┤ Rz(-π/2) ├─────────────░─┤ X ├┤ Sdg ├┤ H ├┤ X ├
      ┌──┴────┴──┐└──┬────┬──┘┌──────────┐ ░ ├───┤└┬───┬┘└───┘└───┘
q_20: ┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(-π/2) ├─░─┤ Y ├─┤ H ├───────────
      ├─────────┬┘   ├────┤   ├──────────┤ ░ ├───┤ ├───┤ ┌───┐     
q_21: ┤ Rz(π/2) ├────┤ √X ├───┤ Rz(-π/2) ├─░─┤ Y ├─┤ H ├─┤ X ├─────
      └──┬────┬─┘ ┌──┴────┴──┐└──────────┘ ░ ├───┤┌┴───┴┐├───┤     
q_22: ───┤ √X ├───┤ Rz(-π/2) ├─────────────░─┤ Y ├┤ Sdg ├┤ H ├─────
       ┌─┴────┴─┐ └──┬────┬──┘┌──────────┐ ░ ├───┤├─────┤├───┤┌───┐
q_23: ─┤ Rz(-π) ├────┤ √X ├───┤ Rz(-π/2) ├─░─┤ Y ├┤ Sdg ├┤ H ├┤ X ├
      ┌┴────────┴┐   ├────┤   ├─────────┬┘ ░ ├───┤└┬───┬┘└───┘└───┘
q_24: ┤ Rz(-π/2) ├───┤ √X ├───┤ Rz(π/2) ├──░─┤ Z ├─┤ H ├───────────
      ├─────────┬┘   ├────┤   ├─────────┤  ░ ├───┤ ├───┤ ┌───┐     
q_25: ┤ Rz(π/2) ├────┤ √X ├───┤ Rz(π/2) ├──░─┤ Z ├─┤ H ├─┤ X ├─────
      └┬────────┤    ├────┤   ├─────────┤  ░ ├───┤┌┴───┴┐├───┤     
q_26: ─┤ Rz(-π) ├────┤ √X ├───┤ Rz(π/2) ├──░─┤ Z ├┤ Sdg ├┤ H ├─────
       └─┬────┬─┘ ┌──┴────┴─┐ └─────────┘  ░ ├───┤├─────┤├───┤┌───┐
q_27: ───┤ √X ├───┤ Rz(π/2) ├──────────────░─┤ Z ├┤ Sdg ├┤ H ├┤ X ├
         └────┘   └─────────┘              ░ └───┘└─────┘└───┘└───┘

In [132]:
procedure.metadata

{'dim': 2, 'backend': 'aer_simulator(fake_quito)', 'rostring': ['1', '1']}